In [ ]:
import pandas as pd
import numpy as np
import math
import subprocess
from datetime import date, time, datetime, timedelta
from IPython.display import display
import calendar
import bandit.main as bd

In [ ]:
def df_search(df_in, cohort):
    if not cohort in df_in:
        return df_in

    df_class = df_bandit_class(df_in, cohort)
    print(df_class)
    next_class = df_class.iloc[0, 0]
    if df_in['Complete'].isnull().sum() > 0:
        df_filtered = df_in[df_in[cohort] == next_class]
    else:
        df_filtered = df_in

    return df_filtered

In [ ]:
def set_start(df):
    goal_minutes = df['Goal']
    final = datetime.combine(date.today(), time(0, 0)) + timedelta(minutes=goal_minutes)
    return final

In [ ]:
# timelist = [datetime.combine(date.today(), time(0, 0)) + timedelta(minutes=i) for i in  range(0,1440,5)]
minute_list = [i for i in  range(5,1440,5)]
df = pd.DataFrame({"Start":minute_list})

In [ ]:
currDate = datetime.today()
minute_list = [i for i in  range(5,1440,5)]
df = pd.DataFrame({"Goal":minute_list})
df["Date"] = currDate
df["Complete"] = np.nan
df = df[['Date', 'Goal', 'Complete']]

In [ ]:
subprocess.Popen('StopSmoking.xlsx', shell=True)

In [ ]:
df_excel = pd.read_excel(io='StopSmoking.xlsx')
df = df.append(df_excel).reset_index(drop=True)

In [ ]:
df = bd.reduce(df,'Goal')

In [ ]:
df.tail(10)

In [ ]:
df['Score'] = np.where(df['Complete'] >= 1, df['Goal']**1.2, 0)
df.loc[df['Complete'].isnull(), 'Score'] = np.NaN

In [ ]:
df_quality = bd.get_quality(df['Score'])

In [ ]:
df = df.join(df_quality).sort_values('Quality', ascending=False)

In [ ]:
val_df = bd.set_cohorts(df, 'Goal')
df = df.merge(val_df, on='Goal')

In [ ]:
currDay = calendar.day_name[currDate.weekday()]
df_today = df
df_today['weekday'] = df_today['Date'].dt.day_name()
df_today = df_today[df_today['weekday'] == currDay]
today_sum = df_today['Complete'].notnull().sum()
if today_sum >= 10:
    df = df_today

In [ ]:
df.sort_values('Date', ascending=True).tail(10)

In [ ]:
df = df.sort_values('Quality', ascending=False)
df.head(10)

In [ ]:
df_step1 = df_search(df,'cohort1')
df_step1

In [ ]:
df_step2 = df_search(df_step1,'cohort2')
df_step2

In [ ]:
df_step3 = df_search(df_step2,'cohort3')
df_step3

In [ ]:
df = df_bandit_class(df_step3, 'Goal')
df['CanStart'] = df.apply(set_start, axis=1).dt.strftime('%I:%M %p')
df